# Tutorial for DRUMBEAT: Dynamically Resolved Universal Model for BayEsiAn network Tracking

In this tutorial, we will demonstrate how to use **DRUMBEAT** package to investigate the probabilistic relationship between variables of interest in temporally resolved manner.

**Step 1: Input Data**

Obtain the temporally resolved data of your choice from MD simulation trajectories.  
In this tutorial, we use intra-protein residue contact information extracted using [GetContacts](https://getcontacts.github.io/).

Each file contains all interactions formed between residues in each frame of the trajectory.  
Each line in the individual `.tsv` file represents **one interaction in one frame**.


In [ ]:
files=['md1.tsv'  ,'md2.tsv'  ,'md3.tsv'  ,'md4.tsv'  ,'md5.tsv']

**Step 2: Constructing universal Bayesian Network.** 

The datasets, provided in this tutorial, can be processed on a standard PC. For large datasets, we do **not** recommend running this step on a standard PC.   
A high-performance computing cluster is strongly recommended to ensure reasonable computation time and resource management.


Import the following modules and make sure to append the path to **DRUMBEAT** and **BaNDyT** folders on your local machine.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('home/drumbeat/src')
sys.path.append('home/bandyt/')
import drumbeat as db
import bandyt as bd

The next command will convert contact data obtained from **GetContacts** into binary matrices.  
Each **column** will represent a contact between two residues (e.g., `R131_L167`), and each **row** will correspond to a frame from the MD simulation trajectory.

Contacts are encoded in a binary format:  
- `1` indicates the contact is present  
- `0` indicates the contact is absent  

The residue contact files provided in this tutorial are **1000 frames** long, and loading them will take approximately **15–20 seconds**.

In [ ]:
MD=db.loadtrajensemble(files)

**Optional:** We recommend performing **feature selection** to retain the most informative variables using pairwise Mutual Information (MI) analysis. By default, only variables (in this case, residue contacts) with a mutual information value greater than 0.05 bits with at least one other variable in each dataset will be retained. The feature selection process is parallelized, and the number of processors used can be specified by the user (default: **4**). For the provided tutorial datasets, feature selection typically takes **2–3 minutes** to complete.

In [ ]:
[m.MIfeatureselect(th=0.05,numproc=4) for m in MD]

After loading the input files and performing optional feature selection, the next step is to create the **universal dataset**. To reduce computation time during Bayesian Network construction, we recommend randomly downsampling the number of trajectory frames used from each dataset. For optimal network resolution, the final dataset should ideally contain **at least** as many data points (frames) as variables (i.e., forming an *N × N* matrix), to ensure proper reconstruction. In this tutorial, we use a sample size of 200 frames per trajectory, resulting in a universal dataset of **1000 frames**.

In [ ]:
uMD=db.getuniversaldataset(MD,samplesize=200)

Using the universal dataset, run **BaNDyT** to compute the **Bayesian Network universal graph**.  
In this tutorial, the dataset contains **33 variables (contacts)** and **1000 data points (trajectory frames)**. The computation should take approximately **5 minutes** using default settings. To accelerate the computation, you can switch to the C-based backend if it has been compiled.  
To do this, change the following line:
```python
srch = bd.search(dt)
```
to 
```python
srch = bd.search(dt, ofunc=bd.cmu)
```
Make sure the C package is compiled. For setup instructions, refer to the [BaNDyT GitHub repository](https://github.com/bandyt-group/bandyt).

In [ ]:
dt=bd.loader(np.vstack((uMD.labels,uMD.traj.astype(int))))
srch=bd.search(dt)
srch.restarts(nrestarts=50)
srch.dot(filename='universalgraph')

**Step 3: Reconstructing Individual Temporally Resolved Graphs from the Universal Graph**

This part of the algorithm can be run on a standard PC.  
The number of processors used can be defined by the user (default: **4**).

Using the generated universal graph (in `.dot` format) and the initially loaded MD dataset, obtain **DRUMBEAT** objects for each trajectory.  
For the demo trajectories provided in this tutorial, this step takes approximately **1 minute** to complete.

In [ ]:
dotfile='universalgraph.dot'
D=db.gettrajdbns(MD,bn_dot=dotfile,windowlist=[150,300],nprocs=4)

To analyze node importance in the reconstructed graph, we can sort and inspect nodes by their **weighted degree** for trajectory 0 (`md1.tsv`).

In [ ]:
D[0].nodes[D[0].wdsort]

To visualize the top 10 nodes with the highest weighted degree in trajectory 0, run the following command:

In [ ]:

traj=0

fig,ax=plt.subplots(figsize=(12,8))
[ax.plot(x) for x in D[traj].wdegree[D[traj].wdsort][:10]]

ax.set_xlabel('Trajectory Frame',fontsize=28)
ax.set_ylabel('Weighted Degree [bits]',fontsize=24)
ax.set_title('Top 10 Weighted Degree Contacts',fontsize=30)
ax.legend(D[0].nodes[D[traj].wdsort][:10],fontsize=18,loc=2)

ax.set_ylim(-0.1,1.1)
ax.tick_params(labelsize=22)

fig.tight_layout()
fig.savefig('top10_weighteddegree.png')
